In [1]:
import pandas as pd

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import streamlit as st
@st.cache_data(show_spinner=True)
def get_fii_table():
    url = "https://www.fundsexplorer.com.br/ranking"

    options = Options()
    # options.add_argument("--headless=new")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(url)
        time.sleep(2)  # espera inicial

        # --- 0) Tenta fechar/aceitar o cookie banner por vários caminhos ---
        cookie_selectors = [
            'button[data-test="accept"]',
            'button#hs-eu-confirmation-button',             # exemplos comuns
            'button[aria-label*="aceitar"]',
            'button:contains("Aceitar")',                   # fallback textual (pode não funcionar no CSS)
            'div#hs-en-cookie-confirmation-buttons-area button',
            'button:contains("Aceitar todos")'
        ]
        # tentar por texto também (mais confiável)
        texts_to_try = ["Aceitar todos", "Aceitar", "OK", "Fechar"]

        # 1) tentar clique por seletores diretos
        for sel in cookie_selectors:
            try:
                els = driver.find_elements(By.CSS_SELECTOR, sel)
                if els:
                    for e in els:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", e)
                            driver.execute_script("arguments[0].click();", e)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    # se algum botão foi clicado, pausa e tenta prosseguir
                    time.sleep(0.6)
            except Exception:
                pass

        # 2) tentar clicar por texto (procura por botões/links)
        for txt in texts_to_try:
            try:
                candidates = driver.find_elements(By.XPATH, f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}') or //a[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}')]]")
                if candidates:
                    for c in candidates:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", c)
                            driver.execute_script("arguments[0].click();", c)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    time.sleep(0.6)
            except Exception:
                pass

        # 3) se ainda existir o elemento de cookie conhecido, remove via JS (forçado)
        try:
            driver.execute_script("""
                var el = document.getElementById('hs-en-cookie-confirmation-buttons-area');
                if (el) { el.remove(); }
                var el2 = document.querySelector('[id^=hs-en-cookie]'); if(el2) el2.remove();
                var overlays = document.querySelectorAll('.cookie, .cookies, .hs-cookie-banner'); 
                overlays.forEach(e => e.remove());
            """)
            time.sleep(0.4)
        except Exception:
            pass

        # --- 4) abrir o menu de colunas (scroll + click via JS para evitar intercept) ---
        botao_colunas = wait.until(EC.presence_of_element_located((By.ID, "colunas-ranking__select-button")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", botao_colunas)
        try:
            # preferencial: click via JS para evitar intercept
            driver.execute_script("arguments[0].click();", botao_colunas)
        except Exception:
            # fallback: webdriver click
            botao_colunas.click()
        time.sleep(0.6)

        # --- 5) clicar em "todos" (label) ---
        label_todos = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'label[for="colunas-ranking__todos"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", label_todos)
        try:
            driver.execute_script("arguments[0].click();", label_todos)
        except Exception:
            label_todos.click()
        time.sleep(1.0)  # deixa o JS atualizar a tabela

        # --- 6) esperar a tabela estar populada ---
        def tabela_populada(driver):
            try:
                rows = driver.find_elements(By.CSS_SELECTOR, ".default-fiis-table__container__table tbody tr")
                # contar linhas não-vazias
                count = 0
                for r in rows:
                    tds = r.find_elements(By.TAG_NAME, "td")
                    if any(td.text.strip() for td in tds):
                        count += 1
                return count > 5  # ajuste se precisar
            except:
                return False

        wait.until(tabela_populada)

        # --- 7) pegar HTML da tabela ---
        tabela = driver.find_element(By.CSS_SELECTOR, ".default-fiis-table__container__table")
        html = tabela.get_attribute("outerHTML")
        df = pd.read_html(html)[0]
        return df

    finally:
        driver.quit()


#Call the function to test
# df_fiis = get_fii_table()
# today_str = pd.Timestamp.today().strftime('%Y-%m-%d')
# df_fiis.to_parquet(f'df_fiis_{today_str}.parquet', index=False)



2025-12-20 03:16:44.184 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-20 03:16:44.186 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-20 03:16:44.187 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-20 03:16:44.535 
  command:

    streamlit run C:\Users\danie\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-20 03:16:44.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-20 03:16:44.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-20 03:16:44.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-20 03:16:45.042 Thread '

In [9]:
df_fiis = pd.read_parquet('df_fiis.parquet')

In [10]:
#show all columns df_fiis
pd.set_option('display.max_columns', None)
df_fiis.head()

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) média,DY (6M) média,DY (12M) média,DY Ano,Variação Preço,Rentab. Período,Rentab. Acumulada,Patrimônio Líquido,VPA,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. Período,Rentab. Patr. Acumulada,Quant. Ativos,Volatilidade,Num. Cotistas,Tax. Gestão,Tax. Performance,Tax. Administração
0,AAGR11,Indefinido,12299,"13.345,09",NaN,146,"1,59 %","1,59 %","6,32 %","16,17 %","0,53 %","1,05 %","1,35 %","13,54 %","4,15 %","5,80 %","26,73 %",None,None,NaN,"0,00 %","0,00 %","0,00 %","0,00 %",0,6127,0,None,None,None
1,AAZQ11,Indefinido,849,"413.207,36",97.0,012,"1,47 %","4,51 %","9,22 %","18,95 %","1,50 %","1,54 %","1,58 %","18,95 %","6,26 %","7,82 %","65,58 %","207.757.234,38",864,98.0,"1,39 %","0,00 %","0,00 %","0,00 %",0,1646,30.064,None,"1,20 % a.a","10,00 % a.a"
2,ABCP11,Shoppings,7808,"76.833,91",72.0,065,"0,86 %","2,39 %","4,93 %","9,71 %","0,80 %","0,82 %","0,81 %","9,71 %","3,52 %","4,41 %","32,77 %","519.221.021,80",11026,71.0,"0,59 %","20,08 %","20,79 %","29,39 %",1,2619,14.130,None,"0,10 % a.a",None
3,AFHI11,Papéis,9456,"798.950,27",100.0,101,"1,06 %","3,21 %","6,72 %","13,71 %","1,07 %","1,12 %","1,14 %","13,71 %","0,17 %","1,23 %","36,47 %","430.678.521,38",9454,100.0,"1,07 %","-0,45 %","0,61 %","6,92 %",14,958,37.566,None,"1,00 % a.a",None
4,AGRX11,Indefinido,802,"264.792,77",75.0,012,"1,49 %","4,49 %","9,22 %","18,06 %","1,50 %","1,54 %","1,51 %","17,12 %","0,62 %","2,12 %","50,57 %","189.942.476,14",1061,76.0,"1,13 %","0,00 %","0,00 %","0,00 %",0,2328,18.900,None,"1,00 % a.a","10,00 % a.a"
